In [1]:
from matplotlib import pyplot as plt 
#from lines_generator import *
#from hough import *
import numpy as np
import cv2
import keras

/home/cordeiro/env/py3im/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Data

In [2]:
%%time

n = 50
Y = list()
patches = list()

#load all patches
for i in range(n):
    Y  += np.load('data/Y'+str(i+1)+'.npy').tolist()
    patches += np.load('data/patches'+str(i+1)+'.npy').tolist()

#add inverse
Y = [[yi,int(not yi)] for yi in Y] 

#to numpy
Y = np.array(Y).astype('uint8')
patches = np.array(patches).astype('uint16')

#add color dimension
patches = np.expand_dims(patches, axis=3)

print(patches.shape, Y.shape)

(19702, 17, 17, 1) (19702, 2)
CPU times: user 842 ms, sys: 56 ms, total: 898 ms
Wall time: 900 ms


__x_train e x_test__

In [3]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(patches, Y, test_size=0.20, random_state=42)

# CNN

In [4]:
from keras import Sequential
from keras.layers import  Conv2D, MaxPooling2D, Flatten, Dense

In [5]:
patche_size = patches.shape[1]
input_shape = (patche_size,patche_size,1)
num_classes = 2

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [7]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Train

In [8]:
model.fit(x_train, y_train,
          epochs=10,
          batch_size=128)

Epoch 1/10
15761/15761 [==============================] - 4s 252us/step - loss: 3.6192 - acc: 0.7691
Epoch 2/10
15761/15761 [==============================] - 4s 251us/step - loss: 3.6335 - acc: 0.7746
Epoch 3/10
15761/15761 [==============================] - 4s 262us/step - loss: 3.6335 - acc: 0.7746
Epoch 4/10
15761/15761 [==============================] - 4s 266us/step - loss: 3.6335 - acc: 0.7746
Epoch 5/10
15761/15761 [==============================] - 4s 249us/step - loss: 3.6335 - acc: 0.7746
Epoch 6/10
15761/15761 [==============================] - 4s 249us/step - loss: 3.6335 - acc: 0.7746
Epoch 7/10
15761/15761 [==============================] - 4s 248us/step - loss: 3.6335 - acc: 0.7746
Epoch 8/10
15761/15761 [==============================] - 4s 259us/step - loss: 3.6335 - acc: 0.7746
Epoch 9/10
15761/15761 [==============================] - 5s 291us/step - loss: 3.6335 - acc: 0.7746
Epoch 10/10
15761/15761 [==============================] - 4s 277us/step - loss: 3.6335 - a

## Evaluate

In [10]:
score = model.evaluate(x_test, y_test, batch_size=128)
score

3941/3941 [==============================] - 0s 94us/step


In [24]:
pred_test = model.predict(x_test)
pred_test[:,1].mean()

1.0